## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Lexington,US,37.9887,-84.4777,25.92,67,43,6.38,scattered clouds,2021-11-29 09:34:05
1,1,Cherskiy,RU,68.7500,161.3000,-19.77,97,100,4.03,overcast clouds,2021-11-29 09:37:33
2,2,Atuona,PF,-9.8000,-139.0333,76.86,81,24,15.82,light rain,2021-11-29 09:37:33
3,3,Dikson,RU,73.5069,80.5464,-15.90,97,99,19.33,overcast clouds,2021-11-29 09:37:33
4,4,Kruisfontein,ZA,-34.0033,24.7314,73.92,53,49,11.39,scattered clouds,2021-11-29 09:37:34


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))



What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
2,2,Atuona,PF,-9.8000,-139.0333,76.86,81,24,15.82,light rain,2021-11-29 09:37:33
6,6,Kapaa,US,22.0752,-159.3190,75.33,66,64,18.43,broken clouds,2021-11-29 09:37:35
14,14,Xuddur,SO,4.1213,43.8894,89.13,33,66,17.98,broken clouds,2021-11-29 09:37:38
15,15,Carnarvon,AU,-24.8667,113.6333,76.78,65,0,30.36,clear sky,2021-11-29 09:35:13
16,16,Bambous Virieux,MU,-20.3428,57.7575,80.73,55,0,10.16,clear sky,2021-11-29 09:37:39
17,17,Gorom-Gorom,BF,14.4439,-0.2361,89.31,9,90,14.09,overcast clouds,2021-11-29 09:37:39
18,18,Rikitea,PF,-23.1203,-134.9692,76.06,75,0,17.45,clear sky,2021-11-29 09:37:39
21,21,Saint-Philippe,RE,-21.3585,55.7679,77.31,81,55,5.91,light rain,2021-11-29 09:37:41
28,28,Vaini,TO,-21.2000,-175.2000,77.56,58,96,17.78,overcast clouds,2021-11-29 09:35:18
35,35,Butaritari,KI,3.0707,172.7902,81.48,79,100,11.32,overcast clouds,2021-11-29 09:37:44


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                192
City                   192
Country                191
Lat                    192
Lng                    192
Max Temp               192
Humidity               192
Cloudiness             192
Wind Speed             192
Current Description    192
Date                   192
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                191
City                   191
Country                191
Lat                    191
Lng                    191
Max Temp               191
Humidity               191
Cloudiness             191
Wind Speed             191
Current Description    191
Date                   191
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Atuona,PF,76.86,light rain,-9.8000,-139.0333,
6,Kapaa,US,75.33,broken clouds,22.0752,-159.3190,
14,Xuddur,SO,89.13,broken clouds,4.1213,43.8894,
15,Carnarvon,AU,76.78,clear sky,-24.8667,113.6333,
16,Bambous Virieux,MU,80.73,clear sky,-20.3428,57.7575,
17,Gorom-Gorom,BF,89.31,overcast clouds,14.4439,-0.2361,
18,Rikitea,PF,76.06,clear sky,-23.1203,-134.9692,
21,Saint-Philippe,RE,77.31,light rain,-21.3585,55.7679,
28,Vaini,TO,77.56,overcast clouds,-21.2000,-175.2000,
35,Butaritari,KI,81.48,overcast clouds,3.0707,172.7902,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   191
Country                191
Max Temp               191
Current Description    191
Lat                    191
Lng                    191
Hotel Name             191
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))